# **Data cleaning**

In [1]:
import pandas as pd

# Table 1 : Orders

## Import & Overview

In [2]:
# orders.csv local
orders_path = 'raw_data/orders.csv'
orders = pd.read_csv(orders_path)

In [3]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


* **created_date** should be datetime type of column
* **state** column can be better named as  **order_status** as it shows the status of an order
* **total_paid** have null values

In [4]:
orders.shape

(226909, 4)

## Duplicates

In [5]:
orders.duplicated().sum()

0

No duplicated rows

## Missing Values

In [6]:
orders.isna().sum()

order_id        0
created_date    0
total_paid      5
state           0
dtype: int64

#### total_paid

In [7]:
perc_total_paid_na = orders.total_paid.isna().sum()*100/orders.shape[0]
perc_total_paid_na

0.0022035265238487677

In [8]:
# alternative to show percentage
orders.total_paid.isna().value_counts(normalize=True)*100

False    99.997796
True      0.002204
Name: total_paid, dtype: float64

In [9]:
f" total_paid column have 5 null values. 5 rows are {perc_total_paid_na.round(4)}% of the total rows of orders table "

' total_paid column have 5 null values. 5 rows are 0.0022% of the total rows of orders table '

As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [10]:
orders = orders.loc[~orders.total_paid.isna()]

In [11]:
orders.isna().sum()

order_id        0
created_date    0
total_paid      0
state           0
dtype: int64

## Data type & Format

In [12]:
orders.dtypes

order_id          int64
created_date     object
total_paid      float64
state            object
dtype: object

#### `order_id`

In [13]:
orders['order_id'] = orders['order_id'].astype(str)

#### `created_date`

In [14]:
orders['created_date'] = pd.to_datetime(orders['created_date'])

#### `state`

In [15]:
orders = orders.rename(columns= {'state':'status'})

In [16]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226904 non-null  object        
 1   created_date  226904 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   status        226904 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 8.7+ MB


## Export cleaned file as csv file

In [17]:
# from google.colab import files

# orders.to_csv("orders_cl.csv", index=False)
# files.download("orders_cl.csv")

In [18]:
# code: local environment
# orders.to_csv("clean_data/orders_cl.csv", index=False)

# Table 2 : Orderlines

## Import & Overview

In [36]:
# orderlines.csv local
orderlines_path = 'raw_data/orderlines.csv'
orderlines = pd.read_csv(orderlines_path)

In [37]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


* `id` , `id_order`, `product_id` should be an object-string datatype
* `date` should be a datetime datatype
* `unit_price` should be a float datatype

## Duplicates

In [38]:
orderlines.duplicated().sum()

0

## Missing Values

In [39]:
orderlines.isna().sum()

id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64

#### `product_id`

In [40]:
orderlines.product_id.unique()

array([0])

all values in product_id column is 0, which is of no sense, so this column can be dropped.

In [41]:
orderlines = orderlines.drop(columns=['product_id'])

## Data Type & Format

In [42]:
orderlines.dtypes

id                   int64
id_order             int64
product_quantity     int64
sku                 object
unit_price          object
date                object
dtype: object

#### `id` , `id_order`

In [43]:
# Convert specific columns to string type
columns_to_convert = ["id", "id_order"]
orderlines[columns_to_convert] = orderlines[columns_to_convert].astype(str)

#### `unit_price`

In [45]:
# commented this line as it gives an error due to value error
# orderlines['unit_price'] = pd.to_numeric(orderlines['unit_price'])

`ValueError: Unable to parse string "1.137.99" at position 6`

This data type conversion give an error (as above), it shows the presence of two decimal values in unit_price column, let's have a look.

In [46]:
two_decimal_unit_price = orderlines.unit_price.str.contains('\d+\.\d+\.\d+')
two_decimal_unit_price.value_counts()

False    257814
True      36169
Name: unit_price, dtype: int64

Let's have a quick investigate to check that's correct. We'll make a DataFrame and then look at a sample to check that all the numbers in the `unit_price` column really have either 2 decimal points

In [47]:
unit_price_problem = orderlines.loc[two_decimal_unit_price]
unit_price_problem.sample(20)

,id,id_order,product_quantity,sku,unit_price,date
132670,1381074,414000,1,APP2087,1.436.79,2017-10-18 15:39:47
272454,1615688,512368,1,APP2254,1.714.59,2018-02-16 06:53:45
200880,1494340,462038,1,APP2067,1.699.00,2017-12-13 17:37:12
18543,1161647,315900,1,PAC1033,2.845.99,2017-01-27 14:37:35
284284,1635834,521338,2,APP2270,3.215.01,2018-03-05 10:36:05
246772,1575458,496628,1,PAC2073,2.598.59,2018-01-23 09:16:00
187647,1476518,455184,1,APP2375,2.010.59,2017-12-04 18:22:36
269316,1610508,510099,1,APP2490,1.159.00,2018-02-12 13:04:52
239375,1563042,489062,1,APP2490,1.139.00,2018-01-16 01:07:19
228354,1544776,484720,1,APP2287,3.188.59,2018-01-07 23:14:47


There are three main patterns for two decimal unit_price and we may remove first decimal for last pattern only, for first two pattern it may affect analysis. Ex. what is maximum unit_price?

**Patterns** :

* ddd.ddd.dd
* dd.ddd.dd
* d.ddd.dd

In [48]:
orderlines.loc[orderlines.unit_price.str.contains('\d{3}\.\d+\.\d+')]

,id,id_order,product_quantity,sku,unit_price,date
36979,1197439,331780,1,NEA0009,159.989.83,2017-03-03 22:29:21


In [49]:
orderlines.loc[orderlines.unit_price.str.contains('\d{2}\.\d+\.\d+')]['unit_price'].count()

33

In [50]:
orderlines.loc[orderlines.unit_price.str.contains('\d\.\d+\.\d+')]['unit_price'].count()

36169

In [51]:
unit_price_problem_perc = unit_price_problem.unit_price.count()*100.0/orderlines.shape[0]
f" 36169 rows are {unit_price_problem_perc.round(4)}% of total rows."

' 36169 rows are 12.3031% of total rows.'

This is a bit of a tricky decision as 12.3% is a significant amount of our data... and we might even end up losing a larger portion of our data than this too. For the moment we will delete the rows for quick, accurate results to show.

Each row of orderlines represents a product in an order. For example, if order number 175 contained 3 seperate products, then order 175 would have 3 rows in orderlines, one row for each of the products. If 2 of those products have 'normal' prices (14.99, 15.85) and 1 has a price with 2 decimal points (1.137.99), we need to remove the whole order and not just the affected row. If we only remove the row with 2 decimal places then any later analysis about products and prices could be misleading.

We therefore need to find the order numbers associated with the rows that have 2 decimal points, and then remove all the associated rows.

In [52]:
# order_id for all orderlines having two decimal value problem
two_decimal_order_ids_list = orderlines.loc[orderlines.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]

# drop all orders- orderlines with two decimal value problem
orderlines = orderlines.loc[~orderlines.id_order.isin(two_decimal_order_ids_list)]

* Let's finally convert unit_price to numeric type

In [53]:
orderlines['unit_price'] = pd.to_numeric(orderlines['unit_price'])

##### Issues with unit_price

unit_price < 0

In [54]:
orderlines.loc[orderlines.unit_price < 0]

,id,id_order,product_quantity,sku,unit_price,date
77008,1268645,365886,1,APP1465,-119.0,2017-06-15 12:48:54


Overview of id_order

In [55]:
orderlines.loc[orderlines.id_order == '365886']

,id,id_order,product_quantity,sku,unit_price,date
77008,1268645,365886,1,APP1465,-119.0,2017-06-15 12:48:54
77009,1268647,365886,1,APP2092,149.0,2017-06-15 12:49:21


How is this order in the orders table?

In [56]:
orders.loc[orders.order_id == '365886']

,order_id,created_date,total_paid,status
66340,365886,2017-06-15 12:51:18,30.0,Cancelled


#### `date`

In [57]:
orderlines['date'] = pd.to_datetime(orderlines['date'])

In [58]:
orderlines = orderlines.rename(columns={'id_order':'order_id'})

In [59]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216250 entries, 0 to 293982
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                216250 non-null  object        
 1   order_id          216250 non-null  object        
 2   product_quantity  216250 non-null  int64         
 3   sku               216250 non-null  object        
 4   unit_price        216250 non-null  float64       
 5   date              216250 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 11.5+ MB


## Export cleaned file as csv file

In [60]:
# # uncomment(below 3 lines) at last to avoid downloading multiple times
# from google.colab import files

# orderlines.to_csv("orderlines_cl.csv", index=False)
# files.download("orderlines_cl.csv")

In [61]:
# code for local machine
# orderlines.to_csv('clean_data/orderlines_cl.csv',index=False)

# Table 3 : Products

## Import & Overview

In [62]:
# products.csv local
products_path = 'raw_data/products.csv'
products = pd.read_csv(products_path)

In [63]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


* **price** and **promo_price** columns should have numeric type
* **in_stock** should be bool type

In [64]:
products.shape

(19326, 7)

## Duplicates

In [67]:
products.duplicated().sum()

8746

In [69]:
products = products.drop_duplicates()

In [71]:
products.duplicated().sum()

0

In [72]:
products.shape

(10580, 7)

## Missing Values

In [73]:
products.isna().sum()

sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64

Manipulating price column can affect statistical analysis, so we will at least do not drop 7 NaN rows in desc and 50 NaN rows in type columns, instead we will replace it with one fixed value.

### `desc`





In [74]:
perc_desc_na = products.desc.isna().sum() * 100.0 / products.shape[0]
f" desc column have 7 null values. 7 rows are {perc_desc_na.round(4)}% of the total rows of products table "

' desc column have 7 null values. 7 rows are 0.0662% of the total rows of products table '

In [75]:
# list of skus for desc have null values
na_desc_sku_list = list(products.loc[products.desc.isna()]['sku'])

# list of order_id for above skus
na_desc_order_id = list(orderlines[orderlines.sku.isin(na_desc_sku_list)]['order_id'])

# no of orderlines for na desc   -- 15
display(orderlines[orderlines.order_id.isin(na_desc_order_id)]['order_id'].count())

# no of orderlines for na desc   -- 11
display(orderlines[orderlines.order_id.isin(na_desc_order_id)]['order_id'].nunique())

15

11

The amonut of missing value in `desc`is not much significant in compare to total data, however fixing it won't make any issue and it is quick. So we will fill it with one value.

In [76]:
# fill with values
products['desc'] = products.desc.fillna("no product description")

In [77]:
# if decide to drop : drop rows
# products = products.loc[~products.desc.isna()]

In [78]:
products.desc.isna().sum()

0

### `price`



In [79]:
perc_price_na = products.price.isna().sum() *100/ products.shape[0]
f" price column have 46 null values. 46 rows are {perc_price_na.round(4)}% of the total rows of products table "

' price column have 46 null values. 46 rows are 0.4348% of the total rows of products table '

`price` column is crucial for analysis and therefore to ensure data quality, we will drop these rows.

In [80]:
products = products.loc[~products.price.isna()]

In [81]:
products.isna().sum()

sku             0
name            0
desc            0
price           0
promo_price     0
in_stock        0
type           50
dtype: int64

### `type`

In [82]:
perc_type_na = products['type'].isna().sum() *100/ products.shape[0]
f" type column have 50 null values. 50 rows are {perc_type_na.round(4)}% of the total rows of products table "

' type column have 50 null values. 50 rows are 0.4747% of the total rows of products table '

In [83]:
# list of skus for type have null values
na_type_sku_list = list(products.loc[products.type.isna()]['sku'])

# list of order_id for above types
na_type_order_id = list(orderlines[orderlines.sku.isin(na_type_sku_list)]['order_id'])

# no of orderlines for na type   -- 249
display(orderlines[orderlines.order_id.isin(na_type_order_id)]['order_id'].count())

# no of orderlines for na type   -- 171
display(orderlines[orderlines.order_id.isin(na_type_order_id)]['order_id'].nunique())

249

171

In [84]:
# fill with value
products['type'] = products['type'].fillna('Unknown')

In [85]:
products = products.rename(columns={'type':'p_type'})

In [86]:
products.isna().sum()

sku            0
name           0
desc           0
price          0
promo_price    0
in_stock       0
p_type         0
dtype: int64

In [87]:
products.shape

(10534, 7)

## Data type & Format

In [88]:
products.dtypes

sku            object
name           object
desc           object
price          object
promo_price    object
in_stock        int64
p_type         object
dtype: object

### `in_stock`

In [89]:
products['in_stock'].unique()

array([1, 0])

In [90]:
products['in_stock'] = products.in_stock.astype(bool)

In [91]:
products.dtypes

sku            object
name           object
desc           object
price          object
promo_price    object
in_stock         bool
p_type         object
dtype: object

### `price`

There are two issues with values in `price`column. Let's have a look.

In [92]:
two_decimal_price = products.price.str.contains('\d+\.\d+\.\d+')
two_decimal_price.value_counts()

False    10157
True       377
Name: price, dtype: int64

In [93]:
three_n_after_decimal_price = products.price.str.contains('\d+\.\d{3,}')
three_n_after_decimal_price.value_counts()

False    9992
True      542
Name: price, dtype: int64

Let's have a quick investigate to check that's correct. We'll make a DataFrame and then look at a sample to check that all the numbers in the `price` column really have either 2 decimal points or 3 decimal places.

In [94]:
price_problem = products.loc[ (two_decimal_price )|(three_n_after_decimal_price)]
price_problem.sample(10)

,sku,name,desc,price,promo_price,in_stock,p_type
17489,APP2491,Apple iPhone X 256GB Space Gray,New Apple iPhone X 256GB Free Space Gray,13.290.011,13.290.011,True,113271716
18926,BEL0302-A,Open - Belkin Audio Adapter (Lightning Charge ...,Refurbished Audio Adapter 35 mm + loaded with ...,349.932,294.604,False,14365395
885,PAC0376,OWC Mercury Elite Pro Dual Thunderbolt + 8TB,RAID outer box 35 inch SATA connection Thunder...,5.609.698,5.549.895,False,11935397
1949,PAC0703,Kingston V300 SSD expansion kit 240GB + 8GB RA...,SSD upgrade kit 240GB + 8GB RAM 1066Mhz + Data...,292.958,2.575.848,False,1433
18838,CAL0010,Station 3 Caldigit Thunderbolt Thunderbolt out...,Thunderbolt external box 3 to magnify your tea...,2.999.905,2.999.905,True,Unknown
15353,REP0309,Microphone Repair iPhone 6 Plus,Repair service including parts and labor for i...,599.906,599.906,False,"1,44E+11"
18833,TRA0044-A,Open - Transcend JetDrive PCIe SSD 820 M13-M15...,Kit 960GB SSD expansion refitted for Macbook P...,7.109.004,5.977.884,False,12215397
16837,APP2311,"Apple MacBook Pro 13 ""Core i5 Touch Bar 31GHz ...",New MacBook Pro 13-inch Core i5 Touch Bar 31 G...,32.455.951,32.455.951,False,"1,02E+12"
19287,REP0410,repair rear camera plus iPhone 6s,Repair service including parts and labor for i...,6.999.003,69.99,False,"1,44E+11"
13181,PAC1510,QNAP TS-251A 8GB RAM NAS,NAS with 8GB of RAM and USB port for direct ac...,4.156.689,3.979.896,False,12175397


In [95]:
price_problem.price.count()

542

In [96]:
price_problem_perc = price_problem.price.count()*100.0/products.shape[0]
f"542 rows are {price_problem_perc.round(4)}% of total rows."

'542 rows are 5.1452% of total rows.'

5.13% is a reasonable amount of our data. However, the price column will be important to understanding discounts, and we should be very trustworthy as we are basing business decisions on it. Therefore, we'll delete these rows.

In [97]:
products = products.loc[~((two_decimal_price)|(three_n_after_decimal_price))]

In [98]:
products.loc[ (two_decimal_price )|(three_n_after_decimal_price)]

,sku,name,desc,price,promo_price,in_stock,p_type


Let's finally convert the column to a numeric datatype

In [99]:
products["price"] = pd.to_numeric(products["price"])

In [100]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9992 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          9992 non-null   object 
 1   name         9992 non-null   object 
 2   desc         9992 non-null   object 
 3   price        9992 non-null   float64
 4   promo_price  9992 non-null   object 
 5   in_stock     9992 non-null   bool   
 6   p_type       9992 non-null   object 
dtypes: bool(1), float64(1), object(5)
memory usage: 556.2+ KB


### `promo_price`

In [101]:
two_decimal_promo_price = products.promo_price.str.contains('\d+\.\d+\.\d+')
two_decimal_promo_price.value_counts()

False    5685
True     4307
Name: promo_price, dtype: int64

In [102]:
three_n_after_decimal_promo_price = products.promo_price.str.contains('\d+\.\d{3,}')
three_n_after_decimal_promo_price.value_counts()

True     9232
False     760
Name: promo_price, dtype: int64

Let's have a quick investigate to check that's correct. We'll make a DataFrame and then look at a sample to check that all the numbers in the `promo_price` column really have either 2 decimal points or 3 decimal places.

In [104]:
promo_price_problem = products.loc[ (two_decimal_promo_price )|(three_n_after_decimal_promo_price)]
promo_price_problem.sample(10)

,sku,name,desc,price,promo_price,in_stock,p_type
18632,STM0105,"STM Studio Case iPad Air / Air 2 / Pro 97 ""/ 2...",STM Studio resistant cover with magnetic closu...,39.95,349.896,False,12635403
18494,LIF0154,LifeProof nüüd iPhone 6s Black Waterproof Case,waterproof case (submersible 2m) 360 degree pr...,89.99,329.894,True,11865403
13692,KEU0002-A,(Open) Kenu Airframe Car Support iPhone 6 / 6S...,portable stand weighs less than 30 grams and c...,19.95,14.278,False,1298
11557,PAC1268,Pack QNAP TS-251 | 4GB RAM | 6TB Seagate Desktop,Pack QNAP TS-251 with 4GB RAM + 6TB (2x3TB) Se...,622.00,4.729.902,False,12175397
12745,IFX0098,iFixit iPhone screen test piece Cable 5S,Cable test function front panel LCD / Digital ...,14.95,119.899,False,21485407
13787,DLL0006-A,"(Open) Dell U2414H Monitor 238 ""IPS Marco Slim",238 inch IPS Monitor Marco Slim.,279.99,2.159.596,False,1298
15927,PAC2050,Second hand - Apple Mac mini Core 2 Duo | 4GB ...,Computer Core i5 Mac mini reconditioned | 4GB ...,699.00,4.555.944,False,"5,44E+11"
12771,IFX0112,iFixit Piece volume button iPhone 5 White,Plastic piece Volume button iPhone 5,5.99,56.906,True,21485407
2939,WDT0276,Western Digital My Passport Ultra 1TB Hard Dri...,WD External Hard Drive Mac and PC USB 3.0 1TB.,84.90,639.896,False,11935397
2713,TUC0208,Tucano Elements Second Skin MacBook Sleeve 12 ...,velvety inner protective case for MacBook 12 i...,24.99,179.903,False,13835403


In [105]:
promo_price_problem.price.count()

9232

In [106]:
promo_price_problem_perc = promo_price_problem.price.count()*100.0/products.shape[0]
f"{promo_price_problem.price.count()} rows are {promo_price_problem_perc.round(4)}% of total rows."

'9232 rows are 92.3939% of total rows.'

That's a lot of wrong data. Over 90% of the data in this column is corrupt. There's no point deleting all of these rows, then we would barely have a products table. Instead, as it's only this column that appears to be very untrustworthy, we will delete the column.

In [107]:
products = products.drop(columns=["promo_price"])

In [108]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9992 entries, 0 to 19325
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sku       9992 non-null   object 
 1   name      9992 non-null   object 
 2   desc      9992 non-null   object 
 3   price     9992 non-null   float64
 4   in_stock  9992 non-null   bool   
 5   p_type    9992 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 478.1+ KB


## Export cleaned file as csv file

In [109]:
# code for working in colab
# from google.colab import files

# products.to_csv("products_cl.csv", index=False)
# files.download("products_cl.csv")

In [110]:
# code for working locally - jupyter notebook
# products.to_csv('clean_data/products_cl.csv',index=False)

# Table 4 : Brands

## Import & Overview

In [111]:
brands_path = 'raw_data/brands.csv'
brands = pd.read_csv(brands_path)

In [112]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   short   187 non-null    object
 1   long    187 non-null    object
dtypes: object(2)
memory usage: 3.1+ KB


In [113]:
brands.shape

(187, 2)

## Duplicates

In [114]:
brands.duplicated().sum()

0

## Missing Values

In [115]:
brands.isna().sum()

short    0
long     0
dtype: int64

## Data types & format

In [116]:
brands.dtypes

short    object
long     object
dtype: object

Ready for next step!!!